In [1]:
%load_ext google.cloud.bigquery

C:\Users\AKIHIKO\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
#Import Libraries
import pydata_google_auth
from google.cloud import bigquery 
import os
import pandas as pd
import numpy as np 
import lightgbm as lgb
import datetime as dt

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

C:\Users\AKIHIKO\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\AKIHIKO\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
credentials = pydata_google_auth.get_user_credentials(['https://www.googleapis.com/auth/bigquery'],)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\AKIHIKO\Desktop\デスクトップ\02 自己学習\09Github\課題\invertible-oath-356510-2e48ff451582.json"

# "Will it snow tomorrow?" - The time traveler asked
The following dataset contains climate information form over 9000 stations accross the world. The overall goal of these subtasks will be to predict whether it will snow tomorrow 13 years ago. So if today is 2022.02.15 then the weather we want to forecast is for the date 2009.02.16. You are suppsed to solve the tasks using Big Query, which can be used in the Jupyter Notebook like it is shown in the following cell. For further information and how to used BigQuery in Jupyter Notebook refer to the Google Docs. 

The goal of this test is, to test your coding knowledge in Python, BigQuery and Pandas as well as your understanding of Data Science. If you get stuck at the first part, you can use the replacement data provided in the second part

In [5]:
%%bigquery 
SELECT
*,
FROM `bigquery-public-data.samples.gsod`
LIMIT 20 


Downloading: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 12.13rows/s]


,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,36010,99999,1929,11,14,42.000000,4,40.200001,4,NaN,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
1,37950,99999,1929,12,28,39.700001,4,38.299999,4,NaN,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
2,38560,99999,1930,6,29,58.500000,4,NaN,<NA>,NaN,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
3,39730,99999,1930,5,26,54.000000,4,49.000000,4,NaN,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
4,36010,99999,1930,4,28,53.700001,4,NaN,<NA>,NaN,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
5,39530,99999,1931,3,18,50.799999,4,47.400002,4,NaN,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
6,38640,99999,1931,5,17,51.500000,4,46.900002,4,NaN,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
7,726810,24131,1931,9,4,79.900002,24,39.000000,8,NaN,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
8,726810,24131,1931,12,2,18.500000,24,13.800000,8,NaN,...,NaN,<NA>,0.0,NaN,True,True,True,True,True,True
9,726810,24131,1931,11,8,54.799999,24,32.200001,8,NaN,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False


## Part 1

### 1. Task
Change the date format to 'YYYY-MM-DD' and select the data from 2005 till 2009 for station numbers including and between 725300 and 726300 , and save it as a pandas dataframe. Note the maximum year available is 2010. 

In [6]:
client = bigquery.Client()

query_str = """
SELECT 
*,
concat(year,'-',month,'-',day) AS date,
FROM `bigquery-public-data.samples.gsod`
WHERE year between 2005 AND 2009
AND station_number between 725300 AND 726300
"""

df = client.query(query_str).to_dataframe()
display(df)


,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado,date
0,725869,99999,2005,7,10,68.400002,5,50.700001,5,NaN,...,<NA>,0.00,NaN,False,False,False,False,False,False,2005-7-10
1,725835,99999,2005,2,11,22.600000,5,16.500000,5,NaN,...,<NA>,0.00,NaN,False,False,False,False,False,False,2005-2-11
2,725868,99999,2005,12,24,28.799999,5,27.700001,5,NaN,...,<NA>,0.00,1.2,False,False,False,False,False,False,2005-12-24
3,725467,99999,2005,6,30,83.800003,5,71.599998,5,NaN,...,<NA>,0.00,NaN,False,False,False,False,False,False,2005-6-30
4,725488,99999,2005,3,24,35.900002,6,26.200001,5,NaN,...,<NA>,0.00,NaN,False,False,False,False,False,False,2005-3-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377779,725330,14827,2006,3,12,57.000000,24,53.400002,24,1014.500000,...,<NA>,0.53,NaN,False,False,False,False,False,False,2006-3-12
377780,725390,14836,2006,2,22,28.600000,24,19.600000,24,1012.599976,...,<NA>,0.00,1.2,False,False,False,False,False,False,2006-2-22
377781,725570,14943,2006,3,25,32.799999,24,26.200001,24,1029.300049,...,<NA>,0.00,NaN,False,False,False,False,False,False,2006-3-25
377782,725500,14942,2006,3,21,30.700001,24,25.600000,24,1020.900024,...,<NA>,0.34,7.1,True,True,True,True,True,True,2006-3-21


### 2. Task 
From here want to work with the data from all stations 725300 to 725330 that have information from 2005 till 2009. 

In [7]:
client = bigquery.Client()

query_str = """
SELECT 
*,
CONCAT(year,'-',month,'-',day) AS date,
FROM `bigquery-public-data.samples.gsod`
WHERE year between 2005 AND 2009
AND station_number between 725300 AND 725330
"""

df = client.query(query_str).to_dataframe()
display(df)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado,date
0,725326,99999,2005,4,18,64.199997,24,46.900002,24,NaN,...,<NA>,0.00,NaN,False,False,False,False,False,False,2005-4-18
1,725326,99999,2005,1,11,31.799999,24,28.600000,24,NaN,...,<NA>,0.00,NaN,False,False,False,False,False,False,2005-1-11
2,725326,99999,2005,9,12,75.500000,24,60.099998,24,NaN,...,<NA>,0.00,NaN,False,False,False,False,False,False,2005-9-12
3,725326,99999,2006,11,29,54.700001,21,50.200001,21,NaN,...,<NA>,0.45,NaN,False,False,False,False,False,False,2006-11-29
4,725326,99999,2006,11,18,36.099998,24,28.600000,24,NaN,...,<NA>,0.00,NaN,False,False,False,False,False,False,2006-11-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18124,725330,14827,2005,2,25,27.900000,24,22.100000,24,1020.900024,...,<NA>,0.00,NaN,True,True,True,True,True,True,2005-2-25
18125,725320,14842,2005,11,9,58.700001,24,46.700001,24,1012.000000,...,<NA>,0.00,NaN,True,True,True,True,True,True,2005-11-9
18126,725330,14827,2005,8,26,72.800003,24,62.599998,24,1016.200012,...,<NA>,0.06,NaN,True,True,True,True,True,True,2005-8-26
18127,725320,14842,2005,6,9,73.199997,24,65.599998,24,1010.599976,...,<NA>,0.00,NaN,True,True,True,True,True,True,2005-6-9


Do a first analysis of the remaining dataset, clean or drop data depending on how you see appropriate. 

In [8]:
#Check data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18129 entries, 0 to 18128
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   station_number                      18129 non-null  Int64  
 1   wban_number                         18129 non-null  Int64  
 2   year                                18129 non-null  Int64  
 3   month                               18129 non-null  Int64  
 4   day                                 18129 non-null  Int64  
 5   mean_temp                           18129 non-null  float64
 6   num_mean_temp_samples               18129 non-null  Int64  
 7   mean_dew_point                      18127 non-null  float64
 8   num_mean_dew_point_samples          18127 non-null  Int64  
 9   mean_sealevel_pressure              16291 non-null  float64
 10  num_mean_sealevel_pressure_samples  16291 non-null  Int64  
 11  mean_station_pressure               993 n

In [9]:
#Check data information
df.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
count,18129.000000,18129.000000,18129.000000,18129.000000,18129.000000,18129.000000,18129.000000,18127.000000,18127.000000,16291.000000,...,0.0,0,17938.000000,565.000000,18129,18129,18129,18129,18129,18129
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,2,2,2,2,2,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,False,False,False,False,False
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,15987,15987,15987,15987,15987,15987
mean,725316.962050,72766.474488,2007.001103,6.527884,15.721000,51.999636,23.574163,41.605108,23.553704,1016.937585,...,NaN,NaN,0.095488,2.865133,NaN,NaN,NaN,NaN,NaN,NaN
std,8.979529,39561.090384,1.413667,3.451484,8.803705,19.392603,1.817230,18.494937,1.857548,6.920370,...,NaN,NaN,0.279290,2.105886,NaN,NaN,NaN,NaN,NaN,NaN
min,725300.000000,3887.000000,2005.000000,1.000000,1.000000,-16.200001,4.000000,-23.500000,4.000000,985.299988,...,NaN,NaN,0.000000,1.200000,NaN,NaN,NaN,NaN,NaN,NaN
25%,725314.000000,14842.000000,2006.000000,4.000000,8.000000,36.299999,24.000000,27.500000,24.000000,1012.799988,...,NaN,NaN,0.000000,1.200000,NaN,NaN,NaN,NaN,NaN,NaN
50%,725316.000000,99999.000000,2007.000000,7.000000,16.000000,53.799999,24.000000,42.599998,24.000000,1016.700012,...,NaN,NaN,0.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,725326.000000,99999.000000,2008.000000,10.000000,23.000000,69.099998,24.000000,57.200001,24.000000,1020.900024,...,NaN,NaN,0.040000,3.900000,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#Delete the column of WBAN (Weather-Bureau-Army-Navy) station numbers from which the data was obtained
df = df.drop(columns=['wban_number'])

#Delete columns due to many missing values
df = df.drop(columns=['mean_station_pressure','max_gust_wind_speed'])

#Delete unrelated columns
df = df.drop(columns=['num_mean_temp_samples','num_mean_dew_point_samples','num_mean_sealevel_pressure_samples','num_mean_station_pressure_samples','num_mean_visibility_samples','num_mean_wind_speed_samples','max_temperature_explicit','min_temperature','min_temperature_explicit'])

#Missing values in the snow_depth column are complemented with 0
df = df.fillna({'snow_depth':0})

#Remove missing row mean_sealevel_pressure 
#Delete [725326]station because it is not snowing.
df = df.dropna(subset = ['mean_sealevel_pressure'])

#Replace TURE and False with 1 and 0
df = df.replace({True:1,False:0})

df["date"] = pd.to_datetime(df["date"])

df = df.sort_values(['station_number', 'date'])

In [11]:
#Assume that only data up to the previous day can be used for snow forecasting.
#This is because data from the day of the forecast cannot be used in tomorrow's snow forecast.

In [12]:
#Split data for each station_number
df_725300 = df[df['station_number']==725300]
df_725305 = df[df['station_number']==725305]
df_725314 = df[df['station_number']==725314]
df_725315 = df[df['station_number']==725315]
df_725316 = df[df['station_number']==725316]
df_725317 = df[df['station_number']==725317]
df_725320 = df[df['station_number']==725320]
df_725327 = df[df['station_number']==725327]
df_725330 = df[df['station_number']==725330]

#Create lag function column with snowfall after 2 days
df_725300['snowlag2'] = df_725300['snow'].shift(-2)
df_725305['snowlag2'] = df_725305['snow'].shift(-2)
df_725314['snowlag2'] = df_725314['snow'].shift(-2)
df_725315['snowlag2'] = df_725315['snow'].shift(-2)
df_725316['snowlag2'] = df_725316['snow'].shift(-2)
df_725317['snowlag2'] = df_725317['snow'].shift(-2)
df_725320['snowlag2'] = df_725320['snow'].shift(-2)
df_725327['snowlag2'] = df_725327['snow'].shift(-2)
df_725330['snowlag2'] = df_725330['snow'].shift(-2)

#Combine all of each station_number data
df2 = pd.concat([df_725300,df_725305,df_725314,df_725315,df_725316,df_725317,df_725320,df_725327,df_725330])

#Remove missing row
df2 = df2.dropna(subset = ['snowlag2'])

C:\Users\AKIHIKO\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\AKIHIKO\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\AKIHIKO\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [13]:
#One-hot encoding of station_number column
df2['station_number'] = df2['station_number'].astype(str)
df2 = pd.get_dummies(df2,columns=['station_number'])

df2['year'] = df2['year'].astype(int)
df2['month'] = df2['month'].astype(int)
df2['day'] = df2['day'].astype(int)

df2 = df2.sort_values(['date'])

### 3. Task
Now it is time to split the data, into a training, evaluation and test set. As a reminder, the date we are trying to predict snow fall for is the following, and hence should constitute your test set.

In [14]:
import datetime as dt

str(dt.datetime.today()- dt.timedelta(days=13*365)).split(' ')[0]


'2009-07-21'

In [15]:
#One year of data up to 2 days prior to forecasting
test_dataset = df2[df2['date'] <= str(dt.datetime.today()- dt.timedelta(days=13*365+2)).split(' ')[0]]
test_dataset = test_dataset[str(dt.datetime.today()- dt.timedelta(days=14*365+2)).split(' ')[0] <  test_dataset['date']]

#Data prior to test data
train_dataset = df2[df2['date'] <= str(dt.datetime.today()- dt.timedelta(days=14*365+2)).split(' ')[0]]


In [16]:
# Split into two types of machine learning models, one for training and one for validation
X_train = train_dataset.drop(['date','snowlag2'], axis=1)  # Make explanatory variables only
y_train = train_dataset['snowlag2'] # Correct class

X_test = test_dataset.drop(['date','snowlag2'], axis=1) # Make explanatory variables only
y_test = test_dataset['snowlag2'] # Correct class

## Part 2
If you made it up to here all by yourself, you can use your prepared dataset to train an Algorithm of your choice to forecast whether it will snow on the following date for each station in this dataset:

In [ ]:
import datetime as dt

str(dt.datetime.today()- dt.timedelta(days=13*365)).split(' ')[0]

You are allowed to use any library you are comfortable with such as sklearn, tensorflow keras etc. 
If you did not manage to finish part one feel free to use the data provided in 'coding_challenge.csv' Note that this data does not represent a solution to Part 1. 

In [17]:
import time
start = time.time()

#Cross-validation considering time series
tscv = TimeSeriesSplit()
tscv.split(X_train, y_train)

param_grid1 = {
    'num_leaves':[5,10,20,30,40],
    'max_depth':[5,10,15,20],
    'n_estimators':[10,25,50,100,200],
    'reg_lambda':[0, 0.001, 0.01, 0.1, 1]

}

#LightGBM is a machine learning framework for gradient boosting based on the decision tree algorithm.
# First adopted due to short model training time and high inference accuracy
lgb_estimator = lgb.LGBMClassifier(random_state=0)

#We want to evaluate False positive rate and Recall in a balanced manner, so 'roc_auc' is used.
gsearch1 = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid1, cv=tscv, scoring = 'roc_auc')
lgb_model1 = gsearch1.fit(X=X_train, y=y_train)

# Display and maintain optimal parameters
best_params = lgb_model1.best_params_
best_score = lgb_model1.best_score_
print(f'Optimum Parameters {best_params}\nScore {best_score}')
print(f'Time{time.time() - start}sec')

C:\Users\AKIHIKO\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:778: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)


Optimum Parameters {'max_depth': 5, 'n_estimators': 50, 'num_leaves': 5, 'reg_lambda': 1}
Score 0.6073345036193695
Time363.51999378204346sec


In [18]:
#Test Result Output
preds = lgb_model1.predict_proba(X_test)
l = preds[:,1]

#After reviewing the predict data, we found that the threshold for the top 10% was 0.17 when ordered by probability of snow.
#0.17 was set as the threshold value.
l_replace = [1 if i > 0.17 else 0 for i in l]

In [19]:
#Test Data Evaluation Results
print('confusion_matrix:',confusion_matrix(y_test, l_replace) )
print('accuracy_score:',accuracy_score(y_test, l_replace))
print('precision_score:',precision_score(y_test, l_replace))
print('recall_score:',recall_score(y_test, l_replace))
print('f1_score:',f1_score(y_test, l_replace))

confusion_matrix: [[2666  281]
 [ 235   95]]
accuracy_score: 0.8425389075373817
precision_score: 0.2526595744680851
recall_score: 0.2878787878787879
f1_score: 0.2691218130311615


In [20]:
#predict whether it will snow tomorrow 13 years ago.
test_dataset2 = df2[df2['date'] == str(dt.datetime.today()- dt.timedelta(days=13*365+2)).split(' ')[0]]

X_test2 = test_dataset2.drop(['date','snowlag2'], axis=1)
y_test2 = test_dataset2['snowlag2'] 

In [21]:
preds2 = lgb_model1.predict_proba(X_test2)
l2 = preds2[:,1]
 
l_replace2 = [1 if i > 0.17 else 0 for i in l2]

test_dataset2['predict_snow']= l_replace2
test_dataset2

C:\Users\AKIHIKO\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,year,month,day,mean_temp,mean_dew_point,mean_sealevel_pressure,mean_visibility,mean_wind_speed,max_sustained_wind_speed,max_temperature,...,station_number_725300,station_number_725305,station_number_725314,station_number_725315,station_number_725316,station_number_725317,station_number_725320,station_number_725327,station_number_725330,predict_snow
2744,2009,7,19,64.800003,55.900002,1020.099976,10.0,3.1,6.0,57.000000,...,0,0,0,1,0,0,0,0,0,0
6422,2009,7,19,64.900002,52.900002,1019.299988,10.0,4.4,8.0,57.000000,...,1,0,0,0,0,0,0,0,0,0
16691,2009,7,19,65.599998,55.200001,1019.799988,10.0,4.3,7.0,57.200001,...,0,0,0,0,0,1,0,0,0,0
5192,2009,7,19,64.500000,53.599998,1020.200012,10.0,2.6,8.0,55.900002,...,0,0,0,0,0,0,1,0,0,0
1991,2009,7,19,62.900002,54.400002,1019.599976,9.9,2.7,14.0,52.000000,...,0,0,0,0,0,0,0,0,1,0
8130,2009,7,19,65.400002,54.900002,1020.200012,10.0,4.0,7.0,55.900002,...,0,0,0,0,1,0,0,0,0,0
8107,2009,7,19,62.799999,55.500000,1019.700012,9.8,2.0,7.0,55.400002,...,0,0,0,0,0,0,0,1,0,0
1836,2009,7,19,68.099998,55.900002,1020.299988,10.0,1.6,5.1,61.000000,...,0,0,1,0,0,0,0,0,0,0
12621,2009,7,19,65.500000,53.799999,1019.400024,10.0,3.0,8.9,57.900002,...,0,1,0,0,0,0,0,0,0,0


In [22]:
#Evaluation Results
print('confusion_matrix:',confusion_matrix(y_test2, l_replace2) )
print('accuracy_score:',accuracy_score(y_test2, l_replace2))
print('precision_score:',precision_score(y_test2, l_replace2))
print('recall_score:',recall_score(y_test2, l_replace2))
print('f1_score:',f1_score(y_test2, l_replace2))

confusion_matrix: [[9]]
accuracy_score: 1.0
precision_score: 0.0
recall_score: 0.0
f1_score: 0.0
